In [1]:
import tensorflow as tf
# #指定使用那块GUP训练
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()
# 设置最大占有GPU不超过显存的70%
config.gpu_options.per_process_gpu_memory_fraction = 0.7 
# # 重点：设置动态分配GPU
config.gpu_options.allow_growth = True
# 创建session时
tf.Session(config=config) 

In [2]:
import numpy as np
from keras.datasets import cifar10
from torch.utils.data import Dataset
import torch.utils.data as Data
class MyDataset(Dataset):
    def __init__(self, imgs, transform=None):
        # super().__init__()
        self.imgs = imgs
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img)
        img=img.reshape([3,32,32])
        return img
import torch
from torch.autograd import Variable
import model
import torch.nn.functional as F
model = model.cifar10(128)
model.load_state_dict(torch.load('./log/default/best-85.pth'))
model.cuda()
def EuclideanDistances(A, B):
    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(), ED.shape[0] * ED.shape[1])


def cal_distance_image_real(images, labels):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=200, shuffle=True)
    y_logits = []
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]
    dict = {}
    all_dis = []
    for i in range(10):
        dict[i] = []
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i] = np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)


def cal_distance_image_fake(images):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=200, shuffle=True)
    y_logits = []
    labels=[]
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = output.data.max(1)[1]
        labels += [i for i in pred.cpu().numpy()]
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]

    dict = {}
    all_dis = []
    for i in range(10):
        dict[i] = []
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i] = np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

import os
if not os.path.isdir('saved_models_{}'.format('cgan')):
    os.mkdir('saved_models_{}'.format('cgan'))
f = open('saved_models_{}/log_collapse1.txt'.format('cgan'), mode='w')
import torch.utils.data as Data
import cv2


from keras.datasets import fashion_mnist,cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, Conv2DTranspose
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.layers import Concatenate, GaussianNoise,Activation
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os
class CGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid = self.discriminator([img, label])

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model([noise, label], valid)
        self.combined.compile(loss=['binary_crossentropy'],
            optimizer=optimizer)

    def build_generator(self):
        model = Sequential()

        model.add(Dense(2 * 2 * 512, activation='relu',input_dim=self.latent_dim))
        model.add(BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.1))
        model.add(Reshape((2, 2, 512)))

        model.add(Conv2DTranspose(256, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.1))


        model.add(Conv2DTranspose(128, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.1))

        model.add(Conv2DTranspose(64, kernel_size=5, strides=2, padding='same'))
        model.add( BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.1))


        model.add(Conv2DTranspose(3, kernel_size=5, strides=2, padding='same', activation='tanh'))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)
    def build_discriminator(self):
        model = Sequential()

        # Conv 1: 16x16x64
        model.add(Conv2D(64, kernel_size=5, strides=2, padding='same' ,input_shape=self.img_shape))
        model.add(BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.1))

        # Conv 2:
        model.add(Conv2D(128, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization(momentum=0.9))
        model.add(LeakyReLU(alpha=0.1))

        # Conv 3:
        model.add(Conv2D(256, kernel_size=5, strides=2, padding='same'))
        model.add( BatchNormalization(momentum=0.9))
        model.add( LeakyReLU(alpha=0.1))

        # Conv 4:
        model.add(Conv2D(512, kernel_size=5, strides=2, padding='same'))
        model.add(BatchNormalization(momentum=0.9))
        model.add( LeakyReLU(alpha=0.1))
        model.summary()

        # FC
        model.add(Flatten())
        img = Input(shape=self.img_shape)
        label = Input(shape=(1,), dtype='int32')
        labels = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))
        discriminator =model(img)

        # Concatenate
        merged_layer = Concatenate()([discriminator, labels])
        discriminator = Dense(512, activation='relu')(merged_layer)

        # Output
        discriminator = Dense(1, activation='sigmoid')(discriminator)

        return Model([img, label], discriminator)




    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (X_test, y_test) = cifar10.load_data()

        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0
        steps = []
        values = []

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                labels = y_train[index * batch_size:(index + 1) * batch_size]

                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, 100))

                # Generate a half batch of new images
                gen_imgs = self.generator.predict([noise, labels])

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch([imgs, labels], valid)
                d_loss_fake = self.discriminator.train_on_batch([gen_imgs, labels], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Condition on labels
                sampled_labels = np.random.randint(0, 10, batch_size).reshape(-1, 1)

                # Train the generator
                g_loss = self.combined.train_on_batch([noise, sampled_labels], valid)

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch,global_step, d_loss[0], 100 * d_loss[1], g_loss))

                sample_num=5000
                if global_step % sample_interval == 0:
                    self.mode_drop(X_test,y_test,sample_num, global_step)


    def mode_drop(self, x_test,y_test,sample_num, global_step):
        r, c = 10, 1000
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise,sampled_labels])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('step:' + str(global_step))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')
if __name__ == '__main__':
    cgan = CGAN()
    cgan.train(epochs=50, batch_size=64, sample_interval=200)


Using TensorFlow backend.
Sequential(
  (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (5): ReLU()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (9): ReLU()
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (12): ReLU()
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 64)        4864      
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 128)         204928    
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 8, 128)         512       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 256)         819456    
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 3.810548, acc.: 17.97%] [G loss: 0.267660]
epoch:0 step:2 [D loss: 0.976476, acc.: 50.00%] [G loss: 3.399368]
epoch:0 step:3 [D loss: 0.777092, acc.: 55.47%] [G loss: 2.964298]
epoch:0 step:4 [D loss: 0.958809, acc.: 31.25%] [G loss: 3.593536]
epoch:0 step:5 [D loss: 0.365670, acc.: 86.72%] [G loss: 3.247061]
epoch:0 step:6 [D loss: 0.660181, acc.: 51.56%] [G loss: 4.713340]
epoch:0 step:7 [D loss: 0.223886, acc.: 93.75%] [G loss: 4.124340]
epoch:0 step:8 [D loss: 0.234675, acc.: 97.66%] [G loss: 4.287393]
epoch:0 step:9 [D loss: 0.099042, acc.: 100.00%] [G loss: 4.902131]
epoch:0 step:10 [D loss: 0.266006, acc.: 92.19%] [G loss: 7.048948]
epoch:0 step:11 [D loss: 0.240475, acc.: 88.28%] [G loss: 5.662122]
epoch:0 step:12 [D loss: 0.172468, acc.: 99.22%] [G loss: 5.737502]
epoch:0 step:13 [D loss: 0.028799, acc.: 100.00%] [G loss: 4.366098]
epoch:0 step:14 [D loss: 0.184768, acc.: 97.66%] [G loss: 5.860831]
epoch:0 step:15 [D loss: 0.278675, acc.: 88.28%] [G los

epoch:0 step:121 [D loss: 0.031016, acc.: 100.00%] [G loss: 4.912096]
epoch:0 step:122 [D loss: 0.010946, acc.: 100.00%] [G loss: 4.630763]
epoch:0 step:123 [D loss: 0.143138, acc.: 92.97%] [G loss: 6.001534]
epoch:0 step:124 [D loss: 0.026390, acc.: 99.22%] [G loss: 6.220888]
epoch:0 step:125 [D loss: 0.088085, acc.: 96.88%] [G loss: 3.734549]
epoch:0 step:126 [D loss: 0.277192, acc.: 88.28%] [G loss: 8.267740]
epoch:0 step:127 [D loss: 0.297989, acc.: 89.84%] [G loss: 7.014802]
epoch:0 step:128 [D loss: 0.058726, acc.: 98.44%] [G loss: 5.453012]
epoch:0 step:129 [D loss: 0.027072, acc.: 99.22%] [G loss: 4.554230]
epoch:0 step:130 [D loss: 0.027423, acc.: 100.00%] [G loss: 3.754258]
epoch:0 step:131 [D loss: 0.091115, acc.: 96.09%] [G loss: 6.006870]
epoch:0 step:132 [D loss: 0.080142, acc.: 96.88%] [G loss: 5.389741]
epoch:0 step:133 [D loss: 0.077796, acc.: 99.22%] [G loss: 4.597342]
epoch:0 step:134 [D loss: 0.041410, acc.: 100.00%] [G loss: 3.141474]
epoch:0 step:135 [D loss: 0.09

...

epoch:49 step:38572 [D loss: 0.000004, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38573 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38574 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38575 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38576 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38577 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38578 [D loss: 0.000001, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38579 [D loss: 0.000007, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38580 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38581 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38582 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38583 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38584 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38585 [D loss: 0.000000,

epoch:49 step:38684 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38685 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38686 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38687 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38688 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38689 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38690 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38691 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38692 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38693 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38694 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38695 [D loss: 0.000001, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38696 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38697 [D loss: 0.000000,

epoch:49 step:38796 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38797 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38798 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38799 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38800 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
##############
[2.1099617  0.91477292 1.05289716 1.02195946 0.82833592 0.96019002
 1.11056359 0.89457065 2.12307529 1.05704471]
##########
epoch:49 step:38801 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38802 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38803 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38804 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38805 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38806 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38807 [D loss: 0.000000, acc.: 10

epoch:49 step:38905 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38906 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38907 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38908 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38909 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38910 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38911 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38912 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38913 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38914 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38915 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38916 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38917 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:38918 [D loss: 0.000002,

epoch:49 step:39016 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:39017 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:39018 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:39019 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:39020 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:39021 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:39022 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:39023 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:39024 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:39025 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:39026 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:39027 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:39028 [D loss: 0.000000, acc.: 100.00%] [G loss: 16.118095]
epoch:49 step:39029 [D loss: 0.000000,